# Settings

In [ ]:
!nvidia-smi

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
!pwd

In [ ]:
!pip install timm

In [ ]:
!git clone https://github.com/moskomule/sam.pytorch.git

# Train

In [ ]:
# arch: resnet10t, resnet50, vit_tiny_patch16_224, vit_small_patch32_224
# optimizer_plus: base, sam

!python main.py --path_to_data ./Data \
                --pretrained_path False \
                --arch resnet10t \
                --optimizer SGD \
                --optimizer_plus base \
                --batch_size 32 \
                --lr 0.0001 \
                --momentum 0.05 \
                --patience 5 \
                --n_epochs 50

In [ ]:
# arch: resnet10t, resnet50, vit_tiny_patch16_224, vit_small_patch32_224
# optimizer_plus: base, sam

!python main.py --path_to_data ./Data \
                --pretrained_path False \
                --arch resnet10t \
                --optimizer SGD \
                --optimizer_plus sam \
                --batch_size 32 \
                --lr 0.0001 \
                --momentum 0.05 \
                --patience 5 \
                --n_epochs 50

In [ ]:
# arch: resnet10t, resnet50, vit_tiny_patch16_224, vit_small_patch32_224
# optimizer_plus: base, sam

!python main.py --path_to_data ./Data \
                --pretrained_path False \
                --arch vit_tiny_patch16_224 \
                --optimizer SGD \
                --optimizer_plus sam \
                --batch_size 32 \
                --lr 0.0001 \
                --momentum 0.05 \
                --patience 5 \
                --n_epochs 50